## Fonction to Raster
Prend en entrée une Satellite Image, un dossier et un nom et la sauve en JP2 dans le dossier considéré

In [6]:
! pip install pyarrow -q -q -q 
! pip install rasterio -q -q -q 
! pip install geopandas -q -q -q
! pip install matplotlib

In [248]:
from datas.datamodule


In [205]:
import sys
sys.path.append('../src')
from satellite_image import SatelliteImage
from utils import *
from plot_utils import *

import yaml
import re
import s3fs
import numpy as np
import matplotlib.pyplot as plt
#import cv2
from PIL import Image as im

from datetime import date
import re
import pyproj
import os
from tqdm import tqdm
from labeler import RILLabeler


In [144]:
path_local_cayenne_data

'/home/onyxia/work/detection-habitat-spontane/notebooks/../data/PLEIADES/2022/GUYANE'

In [11]:
update_storage_access()
environment = get_environment()

root_path = get_root_path()
bucket = environment["bucket"]
path_s3_cayenne_data = environment["sources"]["PLEIADES"][2022]["guyane"]
path_local_cayenne_data = os.path.join(root_path, environment["local-path"]["PLEIADES"][2022]["guyane"])

path_s3_pleiades_data_2022_martinique = environment["sources"]["PLEIADES"][2022]["martinique"]
path_local_pleiades_data_2022_martinique = environment["local-path"]["PLEIADES"][2022]["martinique"]

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})


In [20]:
fs.download(
        rpath=f"{bucket}/{path_s3_cayenne_data}",
        lpath=f"{path_local_cayenne_data}",
        recursive=True)

In [223]:
def to_raster(satellite_image,directory_name,file_name):
    
    data = satellite_image.array
    crs  = satellite_image.crs
    transform = satellite_image.transform
    n_bands = satellite_image.n_bands

    metadata = {
        'dtype': data.dtype,
        'count': n_bands,
        'width': data.shape[2],
        'height': data.shape[1],
        'crs': crs,
        'transform': transform
    }
    
    #print(os.path.exists(directory_name))
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)

    # Save the array as a raster file in jp2 format
    with rasterio.open(directory_name + "/" + file_name, 'w', **metadata) as dst:
        dst.write(data, indexes = np.arange(n_bands)+1)


def write_splitted_images_masks(file_path,output_directory_name,labeler,tile_size,n_bands, dep):
    
    output_images_path  = output_directory_name + "/images"
    output_masks_path  = output_directory_name + "/masks"
    
    if not os.path.exists(output_masks_path):
        os.makedirs(output_masks_path)
        
    list_name = os.listdir(file_path)
    list_path = [file_path + "/" + name for name in list_name]
    
    for path, file_name in zip(tqdm(list_path),list_name): # tqdm ici 

        big_satellite_image = SatelliteImage.from_raster(
            file_path = path,
            dep = None,
            date = None,
            n_bands= 3
        )

        list_satellite_image = big_satellite_image.split(tile_size)

        for i, satellite_image in enumerate(list_satellite_image):
                
                file_name_i = file_name.split(".")[0]+"_"+str(i)
                to_raster(satellite_image,output_images_path,file_name_i + ".tif")
                
                # mask
                mask = labeler.create_segmentation_label(satellite_image) 
                np.save(output_masks_path+"/"+file_name_i+".npy",mask) # save


### Test sur une image

In [214]:
satellite_image = SatelliteImage.from_raster(
        file_path = f"{path_local_cayenne_data}"+ "/ORT_2022072050325085_0352_0545_U22N_16Bits.jp2",
        dep = None,
        date = None,
        n_bands= 4)

print(satellite_image.array.shape)
i = 2
directory_name = "../splitted_data"
file_name = "ORT_2022072050325085_0352_0545_U22N_16Bits"+"_"+str(i)+".tif"

to_raster(satellite_image,directory_name,file_name)


(4, 2000, 2000)


## Test sur l'ensemble des images

- je prends un répertoire en entrée Par exemple Guyane et je lis je split et réécris les images en taille 250
- compter le nombre d'images à traiter et le nombre d'images à l'arrivée.
- Si plusieurs années dispo généraliser le labeller 

In [216]:

# params 
file_path = f"{path_local_cayenne_data}"
tile_size = 250
n_bands = 3
dep ="973"
filename = os.listdir(file_path)[0]
date = datetime.strptime(re.search(r'ORT_(\d{8})', filename).group(1), '%Y%m%d') 
labeler = RILLabeler(date, dep = dep, buffer_size = 10) 
output_directory_name = "../splitted_data"

write_splitted_images_masks(file_path,output_directory_name,labeler,tile_size,n_bands,dep)

# 1 min pour 250 -> 4min pour 1000, ça se tente un peu lionguet mais 

100%|██████████| 240/240 [08:35<00:00,  2.15s/it]


In [221]:
len(os.listdir(output_directory_name+"/masks"))
len(os.listdir(output_directory_name+"/images"))

15360

- a priori si je connais le file path : je connais la date et je connais le labeller => créer labeller en amont ?
- On crée un labeller par date et par territoire concerné 
- On créé les masques et on les sauvegarde

### Les filtres

- Pas de point RIL ... -> out
- trop de nuage  -> out 
- trop de noir -> out (effets de bord)


In [228]:
from filter import is_too_black

### Réadaptation de la classe DeeplaV3module pour la rendre agnostique  au dataset etc..

- modifier les codes de tom pour faire en sorte que les lightning module soient plus résiliants aux modèles ?
- vérifier que le code de Tom marche toujours malgré les modifs 
- normaliser les sortie de data set , image mask pour que ça marche bien aussi 
- il faut que ma classe datamodule soit hermétique au dataset  coisi et au modèle de segmentation choisi et éventuellement à la loss (choisie si tant est qu'elle marche )

## MLFLOW

Je ne vais pas modifier mais proposer une classe plus générique de Segementation module !

1) modification des codes de Tom pour que son train marche quand même

## TO DO

- Trouver des exemples à changements dans Mayotte
- filtrer les patchs mauvais, avant l'écriture..
- PR de Raya